In [197]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

In [17]:
df = pd.read_csv('/Users/avsorokina/en-articles.csv')
df_nyt = pd.read_csv('/Users/avsorokina/en-nyt-articles.csv')
df_ru = pd.read_csv('/Users/avsorokina/ru-articles.csv')

In [221]:
df = df.drop_duplicates()
df_nyt = df_nyt.drop_duplicates()
df_ru = df_ru.drop_duplicates()

In [184]:
lenta_urls = list(set(df_ru[df_ru.source == 'Lenta']['url']))
rbc_urls = list(set(df_ru[df_ru.source == 'RBC']['url']))
rt_urls = list(set(df_ru[df_ru.source == 'RT']['url']))

In [185]:
usa_today_urls = df[df.source == 'USA Today'].reset_index()['url']
wp_urls = df[df.source == 'The Washington Post'].reset_index()['url']
wsj_urls = df[df.source == 'The Wall Street Journal'].reset_index()['url']
nyt_urls = df_nyt['url']

In [65]:
usa_today_urls = [a for a in usa_today_urls if 'restricted' not in a]

In [155]:
url = 'https://www.washingtonpost.com/world/2022/03/26/russia-detention-american-minnesota-teacher-ukraine/'
page = requests.get(url)

In [156]:
soup = BeautifulSoup(page.text, 'html.parser')

In [158]:
table = soup.find_all('div', {'class': 'article-body'})

## USA today

In [124]:
usa_today_texts2 = pd.DataFrame()

In [112]:
usa_today_urls_unique = list(set(usa_today_urls))

In [125]:
for url in usa_today_urls_unique:
    texts = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all('div', {'class': 'gnt_ar_b'})
    for a in table:
        texts.append(a.find('p').text)

    table2 = soup.find_all('p', {'class': 'gnt_ar_b_p'})
    for a in table2:
        if a.text not in texts:
            texts.append(a.text)
    
    usa_today_texts2 = usa_today_texts2.append(pd.Series([url, texts]),  ignore_index=True)
    usa_today_texts2.to_csv('usa_today_texts2.csv')

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1092554760.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usa_today_texts2 = usa_today_texts2.append(pd.Series([url, texts]),  ignore_index=True)
/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1092554760.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usa_today_texts2 = usa_today_texts2.append(pd.Series([url, texts]),  ignore_index=True)


In [127]:
usa_today_texts_clean = usa_today_texts2.loc[~(usa_today_texts2[1].str.len() == 0),:]

In [128]:
usa_today_texts_clean.columns = ['url', 'text']

In [133]:
comb_texts = []
for t in usa_today_texts_clean['text']:
    if len(t) > 1:
        comb_texts.append(' '.join(t))
    else:
        comb_texts.append(t)

In [135]:
usa_today_texts_clean['text'] = comb_texts

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/3526108239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_today_texts_clean['text'] = comb_texts


In [137]:
usa_today_texts_clean.to_csv('usa_today_texts_df.csv')

## The Washington Post

In [166]:
wp_urls_unique = list(set(wp_urls))

In [164]:
wp_texts = pd.DataFrame()

In [167]:
for url in wp_urls_unique:
    texts = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all('div', {'class': 'article-body'})
    for a in table:
        texts.append(a.text)
    
    wp_texts = wp_texts.append(pd.Series([url, texts]),  ignore_index=True)
    wp_texts.to_csv('wp_texts.csv')

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/647009735.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wp_texts = wp_texts.append(pd.Series([url, texts]),  ignore_index=True)
/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/647009735.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wp_texts = wp_texts.append(pd.Series([url, texts]),  ignore_index=True)


In [172]:
wp_texts = wp_texts.loc[~(wp_texts[1].str.len() == 0),:]

In [173]:
wp_texts.columns = ['url', 'text']

In [175]:
comb_texts = []
for t in wp_texts['text']:
    if len(t) > 1:
        comb_texts.append(' '.join(t))
    else:
        comb_texts.append(t)

In [179]:
new_comb_texts = []
for s in comb_texts:
    new_comb_texts.append(" ".join(s.split()))

In [181]:
wp_texts['text'] = new_comb_texts

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/3713873619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wp_texts['text'] = new_comb_texts


In [182]:
wp_texts.to_csv('wp_texts_df.csv')

## NYT

In [186]:
nyt_urls_unique = list(set(nyt_urls))

In [208]:
nyt_texts = pd.DataFrame()

In [209]:
for url in nyt_urls_unique:
    api_url = 'https://www.page2api.com/api/v1/scrape'
    url = url
    payload = {
    "api_key": "cb222cbe6cf15301d8fc6e6b2373b9f5961d1978",
    "url": url,
    "premium_proxy": "us",
    "parse": {
        "description": "section.meteredContent.css-1r7ky0e >> text"
    }
    }

    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    response = requests.post(api_url, data=json.dumps(payload), headers=headers)
    result = json.loads(response.text)
    if result['result'] is not None:
        my_list = [result['result']['description'], url]
        nyt_texts = nyt_texts.append(pd.Series(my_list), ignore_index=True)
        nyt_texts.to_csv('nyt_texts.csv')

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1392798131.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nyt_texts = nyt_texts.append(pd.Series(my_list), ignore_index=True)
/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1392798131.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nyt_texts = nyt_texts.append(pd.Series(my_list), ignore_index=True)


In [210]:
nyt_texts.columns = ['text', 'url']

In [211]:
nyt_texts.dropna().to_csv('nyt_texts_df.csv')

## Lenta

In [195]:
lenta_texts = pd.DataFrame()

In [196]:
for url in lenta_urls:
    texts = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all('p', {'class': 'topic-body__content-text'})
    for a in table:
        texts.append(a.text)
    
    lenta_texts = lenta_texts.append(pd.Series([url, texts]),  ignore_index=True)
    lenta_texts.to_csv('lenta_texts.csv')

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1347338952.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lenta_texts = lenta_texts.append(pd.Series([url, texts]),  ignore_index=True)
/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1347338952.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lenta_texts = lenta_texts.append(pd.Series([url, texts]),  ignore_index=True)


In [198]:
lenta_texts = lenta_texts.loc[~(lenta_texts[1].str.len() == 0),:]

lenta_texts.columns = ['url', 'text']

comb_texts = []
for t in lenta_texts['text']:
    if len(t) > 1:
        comb_texts.append(' '.join(t))
    else:
        comb_texts.append(t)

new_comb_texts = []
for s in comb_texts:
    new_comb_texts.append(" ".join(s.split()))

In [200]:
lenta_texts['text'] = new_comb_texts

In [201]:
lenta_texts.to_csv('lenta_texts_df.csv')

## RBC

In [202]:
rbc_texts = pd.DataFrame()

In [203]:
for url in rbc_urls:
    texts = []
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all('div', {'class': 'article__text article__text_free'})
    for a in table:
        for b in a.find_all('p'):
            texts.append(b.text)

    rbc_texts = rbc_texts.append(pd.Series([url, texts]),  ignore_index=True)
    rbc_texts.to_csv('rbc_texts.csv')

/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1167068182.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rbc_texts = rbc_texts.append(pd.Series([url, texts]),  ignore_index=True)
/var/folders/vd/8vk_qp5n693gjqqjh63dqr5h0000gn/T/ipykernel_82998/1167068182.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rbc_texts = rbc_texts.append(pd.Series([url, texts]),  ignore_index=True)


In [204]:
rbc_texts = rbc_texts.loc[~(rbc_texts[1].str.len() == 0),:]

rbc_texts.columns = ['url', 'text']

comb_texts = []
for t in rbc_texts['text']:
    if len(t) > 1:
        comb_texts.append(' '.join(t))
    else:
        comb_texts.append(t)

new_comb_texts = []
for s in comb_texts:
    new_comb_texts.append(" ".join(s.split()))

In [206]:
rbc_texts['text'] = new_comb_texts

In [207]:
rbc_texts.to_csv('rbc_texts_df.csv')